In [1]:
import json

In [2]:
# run it for each post file
dates=['2021-06-01','2021-09-01','2021-12-01','2022-03-01','2022-06-01']
array = []

for date in dates:
    with open('posts_'+date+'.json', 'r') as f:
        data = json.load(f)
        comment_no = 0
        #  "comments","t1_hmtrrkz", "media", "richtextContent", "document", "c"
        for key in data:
            if key == "comments":
                for comment in data[key]:
                    row=[]
                    row.append(date)
                    for i in range(0,7):
                        # if the data is in a list ie 1. 2. 3. 
                        try: 
                            value=data[key][comment]['media']["richtextContent"]["document"][0]['c'][i]['c'][0]['c'][0]['t']
                            row.append(value)
                        except:
                            # if the data is in a list, but has a non-list sentence first. (Posters often add a preamble)
                            try: 
                                value=data[key][comment]['media']["richtextContent"]["document"][1]['c'][i]['c'][0]['c'][0]['t']
                                row.append(value)
                            except:
                                try: 
                                    # this works if the data is not in a list
                                    value=data[key][comment]['media']["richtextContent"]["document"][i]['c'][0]['t']
                                    row.append(value)
                                except:
                                    pass
                    # remove results with less than 4 lines - these tend to be comments that do not contain salary data (which has 5-7 lines)
                    if len(row)>3:
                        array.append(row)
                    comment_no += 1

In [3]:
import pandas as pd
df=pd.DataFrame(array)
df.columns=['date','title','experience','location','salary','benefits','industry','tech_stack']
df.to_csv('salary_data.csv', index=False)

In [4]:
df

,date,title,experience,location,salary,benefits,industry,tech_stack
0,2021-06-01,"Data Pipeline - Business Side, Sr. Manager",15+ years,"LCOL, US",150K,25%,None,None
1,2021-06-01,Data Engineer,2.5 years as a DE (3 months of that was an int...,100% Remote; company is based out of a low cos...,"130,000USD",10% annual bonus (among other benefits),None,None
2,2021-06-01,Data Engineering Consultant,0.5 years,London,£32K,None,None,None
3,2021-06-01,"It's his first job presumably, I had the same ...",It was tough to live though.,Work doesn't really pay in Europe like it does...,None,None,None,None
4,2021-06-01,ETL Developer,1.5 years,Belarus,$15k USD,None,None,None
...,...,...,...,...,...,...,...,...
330,2022-06-01,Senior Data Engineer,"5, coming on 6",Seattle (but remote mainly),$165k base,"Annual bonus ($25k signing bonus), some equity...",Media,"AWS, Airflow, Spark, some other DevOps tools"
331,2022-06-01,Data Engineer,"1 in role, 2 months in DS internship about 6 m...",Virginia (hcol),102k USD,"Paltry bonus- 10% 401k contrib, full healthcar...",Government,"Python, sql, unfortunately no cloud resources lol"
332,2022-06-01,Data Engineer,8 Years as a BI Dev 2 Years Data Engineer,London UK,"80,000 GBP before Tax",Upto 15% Bonus,Asset Management,"SQL, Azure Data Factory, Python (just a little..."
333,2022-06-01,title: Data Engineer 1,location: Michigan (remote CA company),pay: 145k base,bonus: 10% base/yr + 10k rsu sign on,industry: cannabis,"stack/libs: Python, Pandas, postgresql, boto3,...",None


In [5]:
#create sqlite database
import sqlite3
conn = sqlite3.connect('../df_final.db')

#write to sqlite db
df.to_sql('data_pre_clean', con=conn, if_exists='replace', index=False)

335

In [6]:

# upload to snowflake
import snowflake.connector
from sqlalchemy import create_engine
# from snowflake.connector.pandas_tools import write_pandas 
from snowflake.connector.pandas_tools import pd_writer
from snowflake.sqlalchemy import URL

In [7]:
snowflake_user='devadmin'
snowflake_account='vb69737.us-central1.gcp'
database='NEEDFUL_THINGS',
schema='PUBLIC'
warehouse='COMPUTE_WH'

In [8]:
# Credentials in 1Password
from getpass import getpass
snowflake_password = getpass('Enter the snowflake password (stored in 1password): ')

In [9]:
#snowflake is fussy
df.columns=map(str.upper, df.columns)

In [10]:

from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine

engine = create_engine(URL(
    account = snowflake_account,
    user=snowflake_user,
    password=snowflake_password,
))

connection = engine.connect()
try:
    connection.execute('''USE DATABASE BLOG''')
    connection.execute('''USE WAREHOUSE COMPUTE_WH''')
    df.to_sql('raw_salary_data', connection, if_exists='replace', index = False)
finally:
    connection.close()
    engine.dispose()